# Deep Learning Time Series Forecasting with NHITS and NBEATS

This notebook demonstrates deep learning time series forecasting using NeuralForecast models:
- **NHITS** (Neural Hierarchical Interpolation for Time Series)
- **NBEATS** (Neural Basis Expansion Analysis for Time Series)

## Table of Contents
1. Setup and Data Preparation
2. NHITS: Multi-Scale Forecasting
3. NBEATS: Interpretable Decomposition
4. Comparing NHITS vs NBEATS
5. Working with Exogenous Variables
6. Workflow Integration
7. GPU Acceleration
8. Comparison with Traditional Models

## 1. Setup and Data Preparation

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# py-tidymodels imports
from py_parsnip import nhits_reg, nbeats_reg, prophet_reg, arima_reg
from py_recipes import recipe
from py_workflows import workflow
from py_yardstick import rmse, mae, r_squared, metric_set
from py_rsample import initial_time_split

# Set random seed for reproducibility
np.random.seed(42)

print("✓ Libraries imported successfully")

In [ ]:
# Generate synthetic daily sales data with trend, weekly seasonality, and noise
n_days = 730  # 2 years
dates = pd.date_range(start='2021-01-01', periods=n_days, freq='D')

# Components
trend = np.linspace(100, 150, n_days)  # Linear growth
weekly_seasonality = 20 * np.sin(2 * np.pi * np.arange(n_days) / 7)  # Weekly cycle
noise = np.random.normal(0, 5, n_days)

# Combine components
sales = trend + weekly_seasonality + noise

# Create DataFrame
data = pd.DataFrame({
    'date': dates,
    'sales': sales
})

# Add exogenous variables (for NHITS demonstrations)
data['price'] = 50 + np.random.normal(0, 5, n_days)
data['promo'] = np.random.binomial(1, 0.2, n_days)  # 20% promo days

print(f"Data shape: {data.shape}")
print(f"Date range: {data['date'].min()} to {data['date'].max()}")
data.head()

In [ ]:
# Visualize the data
plt.figure(figsize=(14, 6))
plt.plot(data['date'], data['sales'], alpha=0.7, label='Sales')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.title('Daily Sales Data (with trend and weekly seasonality)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Split into train/test (80/20)
split = initial_time_split(data, prop=0.8)
train_data = split.train_set
test_data = split.test_set

print(f"Train: {len(train_data)} observations ({train_data['date'].min()} to {train_data['date'].max()})")
print(f"Test: {len(test_data)} observations ({test_data['date'].min()} to {test_data['date'].max()})")

## 2. NHITS: Multi-Scale Forecasting

NHITS (Neural Hierarchical Interpolation for Time Series) uses a multi-scale architecture to capture patterns at different time resolutions:
- **Long-term trends** (low frequency)
- **Medium-term patterns** (mid frequency)
- **Short-term dynamics** (high frequency)

**When to use NHITS:**
- Need to include exogenous variables
- Long forecast horizons (30+ steps)
- Multi-scale time series patterns
- Maximum accuracy is priority

In [ ]:
# Create NHITS model
nhits_spec = nhits_reg(
    horizon=7,                     # Forecast 7 days ahead
    input_size=28,                 # Use 28 days of history (4 weeks)
    n_freq_downsample=[8, 4, 1],   # 3 stacks: long/medium/short term
    learning_rate=1e-3,
    max_steps=100,                 # Quick training for demo (use 1000+ in production)
    batch_size=32,
    early_stop_patience_steps=20,
    device='auto',                 # Auto-select GPU if available
    random_seed=42
)

print("NHITS model specification created")
print(f"Model type: {nhits_spec.model_type}")
print(f"Engine: {nhits_spec.engine}")
print(f"Mode: {nhits_spec.mode}")

In [ ]:
# Fit NHITS model (univariate - no exogenous variables)
print("Training NHITS model...")
nhits_fit = nhits_spec.fit(train_data, "sales ~ date")
print("✓ Training complete")

In [ ]:
# Make predictions on test data
nhits_preds = nhits_fit.predict(test_data, type='numeric')
print(f"Predictions shape: {nhits_preds.shape}")
nhits_preds.head()

In [ ]:
# Evaluate on test data
nhits_fit_eval = nhits_fit.evaluate(test_data)

# Extract outputs
nhits_outputs, nhits_coeffs, nhits_stats = nhits_fit_eval.extract_outputs()

# Display metrics
print("\nNHITS Performance Metrics:")
print(nhits_stats[nhits_stats['split'] == 'test'][['split', 'rmse', 'mae', 'r_squared', 'train_time', 'device']])

In [ ]:
# Visualize NHITS predictions
plt.figure(figsize=(14, 6))

# Training data
train_outputs = nhits_outputs[nhits_outputs['split'] == 'train']
plt.plot(train_outputs['dates'], train_outputs['actuals'], alpha=0.5, label='Train Actuals', color='blue')
plt.plot(train_outputs['dates'], train_outputs['fitted'], alpha=0.5, label='Train Fitted', color='cyan')

# Test data
test_outputs = nhits_outputs[nhits_outputs['split'] == 'test']
plt.plot(test_outputs['dates'], test_outputs['actuals'], label='Test Actuals', color='green', linewidth=2)
plt.plot(test_outputs['dates'], test_outputs['fitted'], label='NHITS Forecast', color='red', linewidth=2, linestyle='--')

plt.axvline(test_data['date'].min(), color='black', linestyle=':', alpha=0.5, label='Train/Test Split')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.title('NHITS Forecasting Results')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. NBEATS: Interpretable Decomposition

NBEATS (Neural Basis Expansion Analysis for Time Series) provides interpretable forecasts by decomposing the series into:
- **Trend** (polynomial basis)
- **Seasonality** (harmonic basis)
- **Generic** (learned basis)

**When to use NBEATS:**
- Pure univariate forecasting (no exogenous variables needed)
- Interpretability is important
- Want to understand trend vs seasonality contributions
- Medium forecast horizons (7-30 steps)

In [ ]:
# Create NBEATS model with interpretable stacks
nbeats_spec = nbeats_reg(
    horizon=7,                              # 7-day forecast
    input_size=28,                          # 28-day lookback
    stack_types=['trend', 'seasonality'],   # Interpretable decomposition
    n_polynomials=2,                        # Quadratic trend
    n_harmonics=7,                          # Weekly seasonality (7 harmonics)
    n_blocks=[1, 1],                        # 1 block per stack
    learning_rate=1e-3,
    max_steps=100,
    batch_size=32,
    early_stop_patience_steps=20,
    device='auto',
    random_seed=42
)

print("NBEATS model specification created")
print(f"Stack types: {nbeats_spec.args['stack_types']}")
print(f"Trend complexity: {nbeats_spec.args['n_polynomials']} (polynomial degree)")
print(f"Seasonality complexity: {nbeats_spec.args['n_harmonics']} (harmonics)")

In [ ]:
# Fit NBEATS model (univariate only)
print("Training NBEATS model...")
nbeats_fit = nbeats_spec.fit(train_data, "sales ~ date")
print("✓ Training complete")

In [ ]:
# Make predictions
nbeats_preds = nbeats_fit.predict(test_data, type='numeric')
nbeats_preds.head()

In [ ]:
# Evaluate
nbeats_fit_eval = nbeats_fit.evaluate(test_data)
nbeats_outputs, nbeats_coeffs, nbeats_stats = nbeats_fit_eval.extract_outputs()

print("\nNBEATS Performance Metrics:")
print(nbeats_stats[nbeats_stats['split'] == 'test'][['split', 'rmse', 'mae', 'r_squared', 'train_time', 'device']])

In [ ]:
# Visualize NBEATS predictions
plt.figure(figsize=(14, 6))

# Training data
train_outputs = nbeats_outputs[nbeats_outputs['split'] == 'train']
plt.plot(train_outputs['dates'], train_outputs['actuals'], alpha=0.5, label='Train Actuals', color='blue')
plt.plot(train_outputs['dates'], train_outputs['fitted'], alpha=0.5, label='Train Fitted', color='cyan')

# Test data
test_outputs = nbeats_outputs[nbeats_outputs['split'] == 'test']
plt.plot(test_outputs['dates'], test_outputs['actuals'], label='Test Actuals', color='green', linewidth=2)
plt.plot(test_outputs['dates'], test_outputs['fitted'], label='NBEATS Forecast', color='orange', linewidth=2, linestyle='--')

plt.axvline(test_data['date'].min(), color='black', linestyle=':', alpha=0.5, label='Train/Test Split')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.title('NBEATS Forecasting Results (Interpretable Stacks)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Comparing NHITS vs NBEATS

In [ ]:
# Compare metrics side-by-side
comparison = pd.DataFrame({
    'Model': ['NHITS', 'NBEATS'],
    'RMSE': [
        nhits_stats[nhits_stats['split'] == 'test']['rmse'].values[0],
        nbeats_stats[nbeats_stats['split'] == 'test']['rmse'].values[0]
    ],
    'MAE': [
        nhits_stats[nhits_stats['split'] == 'test']['mae'].values[0],
        nbeats_stats[nbeats_stats['split'] == 'test']['mae'].values[0]
    ],
    'R²': [
        nhits_stats[nhits_stats['split'] == 'test']['r_squared'].values[0],
        nbeats_stats[nbeats_stats['split'] == 'test']['r_squared'].values[0]
    ],
    'Train Time (s)': [
        nhits_stats[nhits_stats['split'] == 'test']['train_time'].values[0],
        nbeats_stats[nbeats_stats['split'] == 'test']['train_time'].values[0]
    ]
})

print("\nModel Comparison:")
print(comparison.to_string(index=False))

# Highlight better model
best_rmse = comparison.loc[comparison['RMSE'].idxmin(), 'Model']
print(f"\n✓ Best RMSE: {best_rmse}")

In [ ]:
# Visual comparison
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# NHITS
test_nhits = nhits_outputs[nhits_outputs['split'] == 'test']
axes[0].plot(test_nhits['dates'], test_nhits['actuals'], label='Actual', color='green', linewidth=2)
axes[0].plot(test_nhits['dates'], test_nhits['fitted'], label='NHITS Forecast', color='red', linewidth=2, linestyle='--')
axes[0].fill_between(test_nhits['dates'], test_nhits['actuals'], test_nhits['fitted'], alpha=0.2, color='red')
axes[0].set_title(f'NHITS - RMSE: {comparison.loc[0, "RMSE"]:.2f}, R²: {comparison.loc[0, "R²"]:.4f}')
axes[0].set_ylabel('Sales')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# NBEATS
test_nbeats = nbeats_outputs[nbeats_outputs['split'] == 'test']
axes[1].plot(test_nbeats['dates'], test_nbeats['actuals'], label='Actual', color='green', linewidth=2)
axes[1].plot(test_nbeats['dates'], test_nbeats['fitted'], label='NBEATS Forecast', color='orange', linewidth=2, linestyle='--')
axes[1].fill_between(test_nbeats['dates'], test_nbeats['actuals'], test_nbeats['fitted'], alpha=0.2, color='orange')
axes[1].set_title(f'NBEATS - RMSE: {comparison.loc[1, "RMSE"]:.2f}, R²: {comparison.loc[1, "R²"]:.4f}')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Sales')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Working with Exogenous Variables (NHITS Only)

NHITS supports exogenous variables, while NBEATS is univariate only.

In [ ]:
# Fit NHITS with exogenous variables
print("Training NHITS with exogenous variables (price, promo)...")
nhits_exog_fit = nhits_spec.fit(train_data, "sales ~ price + promo + date")
print("✓ Training complete")

# Predict
nhits_exog_preds = nhits_exog_fit.predict(test_data, type='numeric')

# Evaluate
nhits_exog_fit_eval = nhits_exog_fit.evaluate(test_data)
_, _, nhits_exog_stats = nhits_exog_fit_eval.extract_outputs()

print("\nNHITS with Exogenous Variables:")
print(nhits_exog_stats[nhits_exog_stats['split'] == 'test'][['split', 'rmse', 'mae', 'r_squared']])

In [ ]:
# NBEATS with exogenous variables (will warn)
print("\nTrying NBEATS with exogenous variables (will warn and ignore them)...")
nbeats_exog_fit = nbeats_spec.fit(train_data, "sales ~ price + promo + date")
print("\n↑ Notice the warning above: NBEATS ignores exogenous variables")

## 6. Workflow Integration

DL models integrate seamlessly with py_recipes and py_workflows.

In [ ]:
# Create recipe with time series preprocessing
from py_recipes import all_numeric_predictors, step_lag, step_rolling, step_normalize

dl_recipe = (
    recipe(train_data, "sales ~ date")
    .step_lag(['sales'], lags=[1, 7])  # 1-day and 1-week lags
    .step_rolling(['sales'], window=7, stats=['mean'])  # 7-day rolling average
    .step_normalize(all_numeric_predictors())  # Normalize features
)

print("Recipe created with time series preprocessing")

In [ ]:
# Create workflow
nhits_workflow = (
    workflow()
    .add_recipe(dl_recipe)
    .add_model(nhits_spec)
)

print("Workflow created")

# Note: Due to lag features, we lose some initial rows
# Use data after sufficient history
train_subset = train_data.iloc[7:]  # Skip first 7 days
test_subset = test_data.iloc[7:]

# Fit workflow
print("\nTraining workflow...")
nhits_wf_fit = nhits_workflow.fit(train_subset)
print("✓ Workflow training complete")

# Predict
nhits_wf_preds = nhits_wf_fit.predict(test_subset)
print(f"\nWorkflow predictions shape: {nhits_wf_preds.shape}")

## 7. GPU Acceleration

Check GPU availability and compare training speed.

In [ ]:
# Check available devices
from py_parsnip.utils import detect_available_devices, get_optimal_device

devices = detect_available_devices()
optimal = get_optimal_device()

print("Available devices:")
for device in devices:
    print(f"  - {device}")
print(f"\nOptimal device: {optimal}")

if 'cuda' in devices:
    print("\n✓ GPU available - training will be 10-50x faster!")
elif 'mps' in devices:
    print("\n✓ Apple Silicon GPU (MPS) available - training will be 5-15x faster!")
else:
    print("\n⚠ No GPU available - using CPU (slower but works)")

In [ ]:
# Compare CPU vs GPU training time (if GPU available)
import time

# CPU model
nhits_cpu = nhits_reg(horizon=7, input_size=28, max_steps=100, device='cpu', random_seed=42)
start = time.time()
nhits_cpu.fit(train_data.iloc[:200], "sales ~ date")  # Subset for faster demo
cpu_time = time.time() - start
print(f"CPU training time: {cpu_time:.2f} seconds")

# GPU model (if available)
if optimal != 'cpu':
    nhits_gpu = nhits_reg(horizon=7, input_size=28, max_steps=100, device=optimal, random_seed=42)
    start = time.time()
    nhits_gpu.fit(train_data.iloc[:200], "sales ~ date")
    gpu_time = time.time() - start
    print(f"{optimal.upper()} training time: {gpu_time:.2f} seconds")
    print(f"\nSpeedup: {cpu_time / gpu_time:.1f}x faster on {optimal.upper()}")
else:
    print("\nGPU not available - skipping GPU benchmark")

## 8. Comparison with Traditional Models

Compare NHITS/NBEATS with Prophet and ARIMA.

In [ ]:
# Train Prophet
print("Training Prophet...")
prophet_spec = prophet_reg()
prophet_fit = prophet_spec.fit(train_data, "sales ~ date")
prophet_fit_eval = prophet_fit.evaluate(test_data)
_, _, prophet_stats = prophet_fit_eval.extract_outputs()
print("✓ Prophet complete")

# Train ARIMA
print("\nTraining ARIMA...")
arima_spec = arima_reg(non_seasonal_ar=1, non_seasonal_differences=1, non_seasonal_ma=1)
arima_fit = arima_spec.fit(train_data, "sales ~ date")
arima_fit_eval = arima_fit.evaluate(test_data)
_, _, arima_stats = arima_fit_eval.extract_outputs()
print("✓ ARIMA complete")

In [ ]:
# Compare all models
all_models = pd.DataFrame({
    'Model': ['NHITS', 'NBEATS', 'Prophet', 'ARIMA'],
    'RMSE': [
        nhits_stats[nhits_stats['split'] == 'test']['rmse'].values[0],
        nbeats_stats[nbeats_stats['split'] == 'test']['rmse'].values[0],
        prophet_stats[prophet_stats['split'] == 'test']['rmse'].values[0],
        arima_stats[arima_stats['split'] == 'test']['rmse'].values[0]
    ],
    'MAE': [
        nhits_stats[nhits_stats['split'] == 'test']['mae'].values[0],
        nbeats_stats[nbeats_stats['split'] == 'test']['mae'].values[0],
        prophet_stats[prophet_stats['split'] == 'test']['mae'].values[0],
        arima_stats[arima_stats['split'] == 'test']['mae'].values[0]
    ],
    'R²': [
        nhits_stats[nhits_stats['split'] == 'test']['r_squared'].values[0],
        nbeats_stats[nbeats_stats['split'] == 'test']['r_squared'].values[0],
        prophet_stats[prophet_stats['split'] == 'test']['r_squared'].values[0],
        arima_stats[arima_stats['split'] == 'test']['r_squared'].values[0]
    ]
})

# Sort by RMSE
all_models = all_models.sort_values('RMSE').reset_index(drop=True)

print("\n" + "="*60)
print("ALL MODELS COMPARISON (sorted by RMSE)")
print("="*60)
print(all_models.to_string(index=False))
print("\n✓ Best model:", all_models.iloc[0]['Model'])

In [ ]:
# Bar chart comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# RMSE comparison
axes[0].bar(all_models['Model'], all_models['RMSE'], color=['red', 'orange', 'blue', 'green'])
axes[0].set_ylabel('RMSE (lower is better)')
axes[0].set_title('Model Comparison: RMSE')
axes[0].grid(True, alpha=0.3, axis='y')

# R² comparison
axes[1].bar(all_models['Model'], all_models['R²'], color=['red', 'orange', 'blue', 'green'])
axes[1].set_ylabel('R² (higher is better)')
axes[1].set_title('Model Comparison: R²')
axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Summary

### Key Takeaways:

1. **NHITS**:
   - Multi-scale architecture (long/medium/short term patterns)
   - Supports exogenous variables
   - Best for complex patterns and long horizons

2. **NBEATS**:
   - Interpretable decomposition (trend + seasonality)
   - Univariate only (no exogenous variables)
   - Best for understanding forecast components

3. **When to Use DL vs Traditional**:
   - **Use DL** (NHITS/NBEATS) when:
     - Large datasets (500+ observations)
     - Complex patterns
     - GPU available
     - Maximum accuracy needed
   
   - **Use Traditional** (Prophet/ARIMA) when:
     - Small datasets (< 500 observations)
     - Simple patterns
     - Interpretability critical
     - Fast inference needed

4. **GPU Acceleration**:
   - CUDA (NVIDIA): 10-50x faster
   - MPS (Apple M1/M2): 5-15x faster
   - CPU: Always available, slower but works

### Next Steps:
- Try hyperparameter tuning with `py_tune`
- Experiment with different architectures
- Test on your own time series data
- Explore hybrid models (NBEATS + XGBoost)